# Multiprocessing

Multiprocessing is a programming concept where multiple processes run independently in their own memory space, each with its own Python interpreter and resources. This is in contrast to multithreading, where multiple threads share the same memory space. Multiprocessing is often used to achieve parallelism, allowing multiple tasks to be executed simultaneously and improving overall program performance. In Python, the `multiprocessing` module provides a way to create and manage processes.

Here's a detailed explanation of multiprocessing with examples:

### 1. Creating Processes:

To create a process in Python, you typically define a function that represents the task you want the process to execute. Then, you create a `Process` object and start it.

```python
from multiprocessing import Process
import time

def print_numbers():
    for i in range(5):
        time.sleep(1)
        print(f"Process: {i}")

# Create a process
process = Process(target=print_numbers)

# Start the process
process.start()

# Wait for the process to finish
process.join()

print("Main process exiting.")
```

In this example, `print_numbers` is a simple function that prints numbers with a delay. The `Process` class is used to create a new process, and the `start` method is called to begin its execution. The `join` method is then used to wait for the process to complete before the main process exits.

### 2. Sharing Data between Processes:

Processes in Python have their own memory space, but it's possible to share data between them using objects like `Value` and `Array` from the `multiprocessing` module.

```python
from multiprocessing import Process, Value
import time

def increment_counter(counter):
    for _ in range(5):
        time.sleep(1)
        with counter.get_lock():
            counter.value += 1
        print(f"Counter: {counter.value}")

# Create a shared counter
counter = Value('i', 0)

# Create a process that increments the counter
process = Process(target=increment_counter, args=(counter,))

# Start the process
process.start()

# Wait for the process to finish
process.join()

print("Main process exiting.")
```

In this example, a shared counter is created using the `Value` class. The `get_lock` method is used to acquire a lock on the counter, ensuring that only one process can modify it at a time.

### 3. Parallel Processing with Pool:

The `Pool` class from the `multiprocessing` module provides a convenient way to parallelize a function over multiple input values.

```python
from multiprocessing import Pool

def square(x):
    return x * x

# Create a Pool with 3 processes
with Pool(processes=3) as pool:
    numbers = [1, 2, 3, 4, 5]
    results = pool.map(square, numbers)

print("Squared results:", results)
```

In this example, the `square` function is applied to a list of numbers using the `map` method of the `Pool` class. The `map` method distributes the work among the specified number of processes.

### 4. Process Communication with Queue:

Processes can communicate with each other using the `Queue` class from the `multiprocessing` module.

```python
from multiprocessing import Process, Queue

def square_numbers(numbers, output):
    for number in numbers:
        output.put(number * number)

# Create a Queue for communication
output_queue = Queue()

# Create a process that squares numbers
numbers_to_square = [1, 2, 3, 4, 5]
process = Process(target=square_numbers, args=(numbers_to_square, output_queue))

# Start the process
process.start()

# Wait for the process to finish
process.join()

# Retrieve results from the Queue
squared_results = []
while not output_queue.empty():
    squared_results.append(output_queue.get())

print("Squared results:", squared_results)
```

In this example, a `Queue` is used for communication between the main process and the child process. The child process squares the numbers and puts the results into the queue, which is then retrieved by the main process.

### 5. Shared Resources with Manager:

The `Manager` class from the `multiprocessing` module allows you to create shared objects like lists, dictionaries, and values.

```python
from multiprocessing import Process, Manager

def update_shared_dict(shared_dict):
    shared_dict['counter'] += 1

# Create a Manager to manage shared resources
with Manager() as manager:
    shared_dict = manager.dict({'counter': 0})

    # Create a process that updates the shared dictionary
    process = Process(target=update_shared_dict, args=(shared_dict,))

    # Start the process
    process.start()

    # Wait for the process to finish
    process.join()

    print("Updated shared dictionary:", shared_dict)
```

In this example, a shared dictionary is created using the `Manager` class. The `update_shared_dict` function is applied to the shared dictionary in a separate process.

These examples cover various aspects of multiprocessing, including creating processes, sharing data between processes, parallel processing, process communication, and using a manager to handle shared resources. Multiprocessing can be especially beneficial for CPU-bound tasks, allowing you to take advantage of multiple processor cores for parallel execution.

Sure, let's break down the problem-solving questions and provide an explanation for each:

### 1. Parallel Matrix Multiplication:

**Problem:**
Implement a program that performs matrix multiplication using multiprocessing. Divide the task among multiple processes to achieve parallelism.

**Explanation:**
Matrix multiplication involves performing a large number of mathematical operations, making it a suitable candidate for parallelization. In this problem, you would divide the matrices into smaller blocks and assign each block multiplication to a separate process. The results are then combined to obtain the final product. This approach utilizes multiple processes to perform computations concurrently, improving performance for large matrices.


```python
from multiprocessing import Process, Manager

def multiply_block(a, b, result, start_row, end_row, start_col, end_col):
    for i in range(start_row, end_row):
        for j in range(start_col, end_col):
            for k in range(len(b)):
                result[i][j] += a[i][k] * b[k][j]

def parallel_matrix_multiplication(a, b, result, num_processes):
    rows_a, cols_a = len(a), len(a[0])
    rows_b, cols_b = len(b), len(b[0])

    # Check if matrix dimensions are valid for multiplication
    if cols_a != rows_b:
        raise ValueError("Invalid matrix dimensions for multiplication")

    # Split the multiplication task among processes
    processes = []
    rows_per_process = rows_a // num_processes

    for i in range(num_processes):
        start_row = i * rows_per_process
        end_row = (i + 1) * rows_per_process if i != num_processes - 1 else rows_a

        process = Process(
            target=multiply_block,
            args=(a, b, result, start_row, end_row, 0, cols_b)
        )
        processes.append(process)
        process.start()

    # Wait for all processes to finish
    for process in processes:
        process.join()

if __name__ == "__main__":
    # Example matrices
    matrix_a = [
        [1, 2],
        [3, 4],
        [5, 6]
    ]

    matrix_b = [
        [7, 8],
        [9, 10]
    ]

    # Initialize result matrix with zeros
    result_matrix = [[0 for _ in range(len(matrix_b[0]))] for _ in range(len(matrix_a))]

    # Number of processes to use
    num_processes = 2

    # Perform parallel matrix multiplication
    parallel_matrix_multiplication(matrix_a, matrix_b, result_matrix, num_processes)

    # Print the matrices and the result
    print("Matrix A:")
    for row in matrix_a:
        print(row)

    print("\nMatrix B:")
    for row in matrix_b:
        print(row)

    print("\nResult Matrix:")
    for row in result_matrix:
        print(row)
```

In this example, we have two matrices (`matrix_a` and `matrix_b`), and we want to compute their product using parallel matrix multiplication. The `multiply_block` function represents the task of multiplying a block of the result matrix. The `parallel_matrix_multiplication` function divides the multiplication task among multiple processes, and each process is responsible for computing a portion of the result matrix.

You can run this script, and it will output the original matrices (`matrix_a` and `matrix_b`) and the result of their multiplication. Feel free to modify the matrices and the number of processes to experiment with different scenarios.



### 2. File Processing with Pool:

**Problem:**
Write a program that processes a large text file line by line. Use a multiprocessing pool to parallelize the processing of each line and analyze the performance improvement.

**Explanation:**
For tasks like line-by-line processing of a large file, multiprocessing can be applied to concurrently process different lines. Using a multiprocessing pool, you can distribute the lines among available processes for parallel execution. This approach helps improve the overall efficiency, especially when dealing with I/O-bound tasks such as reading from a file.


```python
from multiprocessing import Pool

def process_line(line):
    # Process each line (replace this with your custom processing logic)
    return line.upper()

def parallel_file_processing(file_path, num_processes):
    # Read the lines from the file
    with open(file_path, 'r') as file:
        lines = file.readlines()

    # Use a multiprocessing pool to parallelize line processing
    with Pool(num_processes) as pool:
        processed_lines = pool.map(process_line, lines)

    # Write the processed lines back to the file
    with open(file_path + "_processed", 'w') as output_file:
        output_file.writelines(processed_lines)

if __name__ == "__main__":
    # Example file path (replace this with your file path)
    file_path = "sample.txt"

    # Number of processes to use
    num_processes = 2

    # Perform parallel file processing
    parallel_file_processing(file_path, num_processes)

    print(f"File processing completed. Processed file saved as '{file_path}_processed'.")
```

In this example, we have a function `process_line` that represents the processing logic for each line in the file. This function can be customized based on the specific processing you want to apply to each line.

The `parallel_file_processing` function reads lines from a file, uses a multiprocessing pool to parallelize the line processing, and then writes the processed lines back to a new file. The number of processes used is determined by the `num_processes` parameter.

Please replace the `file_path` variable with the path to your input file, and customize the `process_line` function according to your specific requirements. After running this script, you should see a processed file saved with the "_processed" suffix.



### 3. Parallel Image Processing:

**Problem:**
Create a program that applies a filter or transformation to each pixel of an image. Use multiprocessing to parallelize the pixel processing and speed up the image transformation.

**Explanation:**
Image processing often involves operations on individual pixels, making it a suitable candidate for parallelization. By dividing the image into smaller regions and assigning each region to a separate process, you can apply filters or transformations concurrently. This approach leverages the parallel processing capabilities to enhance the performance of image processing tasks.


```python
from multiprocessing import Pool
from PIL import Image

def apply_filter(pixel):
    # Example: Apply a simple filter (replace this with your custom filter)
    return tuple(value * 2 for value in pixel)

def parallel_image_processing(image_path, output_path, num_processes):
    # Open the image
    with Image.open(image_path) as image:
        # Get image dimensions
        width, height = image.size

        # Get pixel data
        pixels = list(image.getdata())

        # Use a multiprocessing pool to parallelize pixel processing
        with Pool(num_processes) as pool:
            processed_pixels = pool.map(apply_filter, pixels)

        # Create a new image with the processed pixel data
        processed_image = Image.new("RGB", (width, height))
        processed_image.putdata(processed_pixels)

        # Save the processed image
        processed_image.save(output_path)

if __name__ == "__main__":
    # Example image path (replace this with your image path)
    image_path = "input_image.jpg"

    # Output path for the processed image
    output_path = "output_image.jpg"

    # Number of processes to use
    num_processes = 2

    # Perform parallel image processing
    parallel_image_processing(image_path, output_path, num_processes)

    print(f"Image processing completed. Processed image saved as '{output_path}'.")
```

In this example, we are using the Python Imaging Library (PIL) module, which is commonly used for working with images. You can install it using:

```bash
pip install Pillow
```

The `apply_filter` function represents the image processing logic for each pixel. This function can be customized based on the specific filter or transformation you want to apply.

The `parallel_image_processing` function opens an image, retrieves pixel data, and uses a multiprocessing pool to parallelize the pixel processing. The processed pixels are then used to create a new image, which is saved to the specified output path.

Replace the `image_path` variable with the path to your input image and customize the `apply_filter` function according to your specific image processing requirements. After running this script, you should see a processed image saved at the specified output path.



### 4. Parallel Web Scraping:

**Problem:**
Build a web scraper that extracts information from multiple web pages simultaneously. Use multiprocessing to fetch and process the web pages concurrently.

**Explanation:**
Web scraping involves making requests to multiple web pages, which can be time-consuming. Using multiprocessing, you can parallelize the scraping process by assigning different URLs to separate processes. This enables fetching and processing of web pages concurrently, reducing the overall time required to collect information from multiple sources.


```python
import requests
from bs4 import BeautifulSoup
from multiprocessing import Pool

def scrape_page(url):
    # Example: Scrape titles of articles from a webpage (replace this with your custom scraping logic)
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'html.parser')
    titles = [title.text.strip() for title in soup.find_all('h2')]
    return titles

def parallel_web_scraping(urls, num_processes):
    # Use a multiprocessing pool to parallelize web scraping
    with Pool(num_processes) as pool:
        scraped_data = pool.map(scrape_page, urls)

    return scraped_data

if __name__ == "__main__":
    # Example list of URLs to scrape (replace this with your URLs)
    urls_to_scrape = [
        'https://example.com/page1',
        'https://example.com/page2',
        'https://example.com/page3'
    ]

    # Number of processes to use
    num_processes = 2

    # Perform parallel web scraping
    scraped_results = parallel_web_scraping(urls_to_scrape, num_processes)

    # Print the scraped results
    for url, titles in zip(urls_to_scrape, scraped_results):
        print(f"Titles from {url}:\n{titles}\n")
```

In this example, we are using the `requests` library to make HTTP requests and the `BeautifulSoup` library for HTML parsing. You can install these libraries using:

```bash
pip install requests beautifulsoup4
```

The `scrape_page` function represents the web scraping logic for a single webpage. This function can be customized based on the specific information you want to extract from each webpage.

The `parallel_web_scraping` function uses a multiprocessing pool to parallelize web scraping across multiple URLs. The list `urls_to_scrape` contains the URLs of the web pages to scrape. The result is a list of scraped data for each URL.

Replace the `urls_to_scrape` variable with your list of URLs and customize the `scrape_page` function according to your specific web scraping requirements. After running this script, you should see the titles or data extracted from the specified web pages.



### 5. Parallel Search:

**Problem:**
Implement a parallel search algorithm, such as parallel binary search or parallel linear search, using multiprocessing to search for an element in a large dataset.

**Explanation:**
Searching algorithms can be parallelized by dividing the dataset into smaller chunks and assigning each chunk to a separate process. Parallel search algorithms can improve the speed of finding elements in large datasets by exploring multiple parts of the dataset concurrently.


```python
from multiprocessing import Pool

def parallel_linear_search(data, target, start, end):
    for i in range(start, end):
        if data[i] == target:
            return i
    return -1

def parallel_search(data, target, num_processes):
    # Split the search task among processes
    processes = []
    data_size = len(data)
    chunk_size = data_size // num_processes

    for i in range(num_processes):
        start_index = i * chunk_size
        end_index = (i + 1) * chunk_size if i != num_processes - 1 else data_size

        process = Pool().apply_async(parallel_linear_search, (data, target, start_index, end_index))
        processes.append(process)

    # Wait for all processes to finish
    for process in processes:
        result = process.get()
        if result != -1:
            return result

    return -1

if __name__ == "__main__":
    # Example list of data (replace this with your data)
    data_to_search = [1, 3, 5, 7, 9, 2, 4, 6, 8, 10]

    # Target value to search for
    target_value = 6

    # Number of processes to use
    num_processes = 2

    # Perform parallel search
    result_index = parallel_search(data_to_search, target_value, num_processes)

    if result_index != -1:
        print(f"Target value {target_value} found at index {result_index}.")
    else:
        print(f"Target value {target_value} not found in the list.")
```

In this example, we're performing a parallel linear search on a list of data using multiprocessing. The `parallel_linear_search` function represents the search logic for a specific range of the list.

The `parallel_search` function divides the search task among multiple processes using the `Pool` class. Each process executes the `parallel_linear_search` function on a specific portion of the list. The result is the index where the target value is found, or -1 if the target value is not present.

Replace the `data_to_search` variable with your list of data and customize the `parallel_linear_search` function or use a different search algorithm based on your specific requirements. After running this script, you should see whether the target value was found and at which index.



### 6. Parallel Sorting:

**Problem:**
Design a program that performs parallel sorting of a large list using multiprocessing. Explore parallel algorithms like parallel merge sort or parallel quicksort.

**Explanation:**
Sorting algorithms, especially those based on divide and conquer, can be parallelized by dividing the dataset and sorting each portion concurrently. Parallel sorting algorithms like parallel merge sort or parallel quicksort use multiprocessing to achieve faster sorting of large datasets.


```python
from multiprocessing import Pool
import random

def parallel_merge_sort(data):
    if len(data) <= 1:
        return data

    mid = len(data) // 2
    left = data[:mid]
    right = data[mid:]

    with Pool(2) as pool:
        left = pool.apply_async(parallel_merge_sort, (left,))
        right = pool.apply_async(parallel_merge_sort, (right,))
        left = left.get()
        right = right.get()

    return merge(left, right)

def merge(left, right):
    merged = []
    left_index = 0
    right_index = 0

    while left_index < len(left) and right_index < len(right):
        if left[left_index] < right[right_index]:
            merged.append(left[left_index])
            left_index += 1
        else:
            merged.append(right[right_index])
            right_index += 1

    merged.extend(left[left_index:])
    merged.extend(right[right_index:])
    return merged

if __name__ == "__main__":
    # Example list of data (replace this with your data)
    data_to_sort = [random.randint(1, 100) for _ in range(10)]

    print("Original Data:", data_to_sort)

    # Perform parallel merge sort
    sorted_data = parallel_merge_sort(data_to_sort)

    print("Sorted Data:", sorted_data)
```

In this example, we're implementing a parallel merge sort using multiprocessing. The `parallel_merge_sort` function recursively divides the data into two halves and applies parallel merge sort to each half. The `merge` function then combines the sorted halves.

Replace the `data_to_sort` variable with your list of data. After running this script, you should see the original data and the sorted data using parallel merge sort. Note that this is a demonstration, and for small datasets, the overhead of parallelization may outweigh the benefits. Parallel sorting is more effective for larger datasets.



### 7. Distributed Task Execution:

**Problem:**
Create a system where tasks are distributed among multiple processes or machines for execution. Use multiprocessing or the `multiprocessing` module to manage the distribution and execution of tasks.

**Explanation:**
Distributed task execution involves breaking down a larger task into smaller sub-tasks and distributing them across multiple processes or machines. The `multiprocessing` module can be used for managing the distribution and coordination of tasks among different processing units.


This problem involves creating a system where tasks are distributed among multiple processes or machines for execution. Below is a simplified example demonstrating a basic distributed computing scenario using Python's multiprocessing module and a client-server model.

```python
from multiprocessing import Process, Manager

def worker_task(task_queue, result_dict):
    while True:
        task = task_queue.get()
        if task is None:
            break  # Exit when the queue is empty and all tasks are processed

        # Perform the task (replace this with your custom task execution logic)
        result = task * 2

        # Store the result in the result dictionary
        result_dict[task] = result

def distribute_tasks(tasks, num_processes):
    # Create a task queue and a result dictionary shared among processes
    task_queue = Manager().Queue()
    result_dict = Manager().dict()

    # Add tasks to the queue
    for task in tasks:
        task_queue.put(task)

    # Add None to signal the end of tasks
    for _ in range(num_processes):
        task_queue.put(None)

    # Create worker processes
    processes = [Process(target=worker_task, args=(task_queue, result_dict)) for _ in range(num_processes)]

    # Start worker processes
    for process in processes:
        process.start()

    # Wait for worker processes to finish
    for process in processes:
        process.join()

    return result_dict

if __name__ == "__main__":
    # Example list of tasks (replace this with your tasks)
    tasks_to_distribute = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

    # Number of processes to use
    num_processes = 3

    # Distribute tasks among processes
    results = distribute_tasks(tasks_to_distribute, num_processes)

    # Print the results
    print("Results:")
    for task, result in results.items():
        print(f"Task: {task}, Result: {result}")
```

In this example, the `worker_task` function represents the task execution logic. The `distribute_tasks` function distributes tasks among worker processes using a shared task queue and a result dictionary. Worker processes take tasks from the queue, perform the tasks, and store the results in the shared dictionary.

Replace the `tasks_to_distribute` variable with your list of tasks, and customize the `worker_task` function with your specific task execution logic. After running this script, you should see the results of the tasks distributed and executed across multiple processes.



### 8. Parallel Monte Carlo Simulation:

**Problem:**
Implement a Monte Carlo simulation that estimates a mathematical value using random sampling. Use multiprocessing to run multiple simulations concurrently, improving the accuracy of the estimate.

**Explanation:**
Monte Carlo simulations involve repeated random sampling to estimate a mathematical value. By running multiple simulations concurrently using multiprocessing, you can obtain more accurate estimates in a shorter amount of time.


This problem involves implementing a parallel Monte Carlo simulation to estimate a mathematical value using random sampling. Here's an example using the estimation of π (pi) as a Monte Carlo simulation:

```python
import random
from multiprocessing import Pool

def monte_carlo_simulation(total_points):
    points_inside_circle = 0

    for _ in range(total_points):
        x, y = random.uniform(-1, 1), random.uniform(-1, 1)
        distance = x**2 + y**2

        if distance <= 1:
            points_inside_circle += 1

    return points_inside_circle

def parallel_monte_carlo_simulation(total_points, num_processes):
    points_per_process = total_points // num_processes

    with Pool(num_processes) as pool:
        results = pool.map(monte_carlo_simulation, [points_per_process] * num_processes)

    total_points_inside_circle = sum(results)
    pi_estimate = 4 * (total_points_inside_circle / total_points)

    return pi_estimate

if __name__ == "__main__":
    # Example total points for the simulation
    total_points_to_simulate = 1000000

    # Number of processes to use
    num_processes = 4

    # Perform parallel Monte Carlo simulation
    pi_estimate = parallel_monte_carlo_simulation(total_points_to_simulate, num_processes)

    print(f"Estimated value of π: {pi_estimate}")
```

In this example, the `monte_carlo_simulation` function simulates random points in a unit square and estimates π by calculating the ratio of points inside a quarter circle to the total points.

The `parallel_monte_carlo_simulation` function uses the `Pool` class from the `multiprocessing` module to parallelize the simulation across multiple processes. Each process runs a separate Monte Carlo simulation, and the results are combined to obtain the final estimate of π.

Feel free to adjust the `total_points_to_simulate` and `num_processes` variables according to your requirements. The more points you simulate, and the more processes you use, the more accurate the estimation will be.



### 9. Parallel Data Analysis:

**Problem:**
Develop a program that analyzes a large dataset, such as calculating statistical measures or aggregating data. Utilize multiprocessing to parallelize the analysis for faster results.

**Explanation:**
Data analysis tasks, such as calculating statistics or aggregating data, can benefit from parallel processing. By dividing the dataset and processing different portions concurrently, multiprocessing can significantly speed up the data analysis process.


This problem involves developing a program that analyzes a large dataset, such as calculating statistical measures or aggregating data. Here's an example using a parallel approach with multiprocessing to speed up the data analysis:

```python
from multiprocessing import Pool

def analyze_data_segment(data_segment):
    # Example: Calculate the sum of the data segment (replace this with your custom analysis logic)
    return sum(data_segment)

def parallel_data_analysis(data, num_processes):
    data_size = len(data)
    segment_size = data_size // num_processes

    with Pool(num_processes) as pool:
        results = pool.map(analyze_data_segment, [data[i:i+segment_size] for i in range(0, data_size, segment_size)])

    total_result = sum(results)
    return total_result

if __name__ == "__main__":
    # Example dataset (replace this with your dataset)
    large_dataset = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15]

    # Number of processes to use
    num_processes = 2

    # Perform parallel data analysis
    result = parallel_data_analysis(large_dataset, num_processes)

    print(f"Result of parallel data analysis: {result}")
```

In this example, the `analyze_data_segment` function represents the analysis logic for a specific segment of the dataset. The `parallel_data_analysis` function divides the dataset into segments and uses multiprocessing to parallelize the analysis. The results from each segment are then combined to obtain the total result.

Replace the `large_dataset` variable with your actual dataset, and customize the `analyze_data_segment` function with your specific data analysis logic. The `num_processes` variable determines how many processes will be used for parallel analysis.

This example demonstrates a simple sum calculation, but you can replace it with more complex statistical measures or any other analysis logic based on your requirements.



### 10. Distributed Computing with Client-Server Model:

**Problem:**
Design a system where a server distributes tasks to multiple client processes for execution. Use multiprocessing and a client-server architecture to achieve distributed computing.

**Explanation:**
Distributed computing involves distributing tasks among multiple processing units in a network. Using multiprocessing in a client-server architecture, the server can distribute tasks to client processes, and each client can independently perform its assigned task, allowing for efficient distributed computing.


This problem involves designing a system where a server distributes tasks to multiple client processes for execution. Here's a simple example using multiprocessing and a client-server model:

**server.py**
```python
from multiprocessing import Process, Queue
import time

def distribute_tasks(task_queue, num_clients):
    # Example: Distribute tasks to client processes (replace this with your custom task distribution logic)
    tasks = [i for i in range(10)]  # Replace this with your list of tasks

    # Distribute tasks to the task queue
    for task in tasks:
        task_queue.put(task)

    # Add None to signal the end of tasks
    for _ in range(num_clients):
        task_queue.put(None)

def server(task_queue, result_queue):
    # Number of client processes
    num_clients = 3

    # Create and start client processes
    client_processes = [Process(target=client, args=(task_queue, result_queue)) for _ in range(num_clients)]
    for process in client_processes:
        process.start()

    # Distribute tasks to client processes
    distribute_tasks(task_queue, num_clients)

    # Wait for client processes to finish
    for process in client_processes:
        process.join()

def client(task_queue, result_queue):
    while True:
        task = task_queue.get()
        if task is None:
            break  # Exit when there are no more tasks

        # Example: Perform the task (replace this with your custom task execution logic)
        result = task * 2

        # Store the result in the result queue
        result_queue.put(result)

if __name__ == "__main__":
    task_queue = Queue()
    result_queue = Queue()

    # Start the server process
    server_process = Process(target=server, args=(task_queue, result_queue))
    server_process.start()

    # Wait for the server process to finish
    server_process.join()

    # Retrieve and print results from the result queue
    print("Results:")
    while not result_queue.empty():
        result = result_queue.get()
        print(result)
```

In this example, the server distributes tasks to client processes through a task queue. The `distribute_tasks` function in the server distributes a list of tasks to the task queue, and each client process executes tasks from the queue. The `None` sentinel value is used to signal the end of tasks.

**Note:** This is a simplified example, and in a real-world scenario, you may need to handle more complex tasks, communication between processes, and error handling. Adjust the code based on your specific requirements.

To run this example, save it in two separate files (e.g., `server.py` and `client.py`) and execute them separately. The server will distribute tasks to client processes, and the clients will execute the tasks and store the results in a result queue. The main program then retrieves and prints the results from the result queue.



### 11. Parallel Genetic Algorithm:

**Problem:**
Implement a parallel version of a genetic algorithm to optimize a solution space. Use multiprocessing to evaluate multiple candidate solutions concurrently.

**Explanation:**
Genetic algorithms involve evaluating multiple candidate solutions in parallel. By using multiprocessing, the fitness evaluation of different candidate solutions can be performed concurrently, accelerating the optimization process.


This problem involves implementing a parallel version of a genetic algorithm to optimize a solution space. Here's a basic example using multiprocessing to evaluate multiple candidate solutions concurrently:

```python
from multiprocessing import Pool
import random

def evaluate_fitness(candidate):
    # Example: Evaluate the fitness of a candidate solution (replace this with your custom fitness function)
    return sum(candidate)

def generate_random_candidate():
    # Example: Generate a random candidate solution (replace this with your custom generation logic)
    return [random.randint(0, 10) for _ in range(5)]

def parallel_genetic_algorithm(population_size, num_generations, num_processes):
    # Generate an initial population of random candidates
    population = [generate_random_candidate() for _ in range(population_size)]

    for generation in range(num_generations):
        # Evaluate the fitness of each candidate in parallel
        with Pool(num_processes) as pool:
            fitness_scores = pool.map(evaluate_fitness, population)

        # Select the top candidates based on fitness scores (replace this with your custom selection logic)
        selected_indices = sorted(range(len(fitness_scores)), key=lambda k: fitness_scores[k], reverse=True)[:population_size]

        # Create the next generation by crossover and mutation (replace this with your custom genetic operations)
        next_generation = []
        for _ in range(population_size):
            parent1 = population[random.choice(selected_indices)]
            parent2 = population[random.choice(selected_indices)]
            crossover_point = random.randint(0, len(parent1))
            child = parent1[:crossover_point] + parent2[crossover_point:]
            mutation_rate = 0.1
            if random.random() < mutation_rate:
                mutation_point = random.randint(0, len(child) - 1)
                child[mutation_point] = random.randint(0, 10)
            next_generation.append(child)

        # Replace the old population with the new generation
        population = next_generation

    # Return the best candidate after the specified number of generations
    best_candidate = population[fitness_scores.index(max(fitness_scores))]
    return best_candidate

if __name__ == "__main__":
    # Example parameters
    population_size = 10
    num_generations = 5
    num_processes = 2

    # Run the parallel genetic algorithm
    best_solution = parallel_genetic_algorithm(population_size, num_generations, num_processes)

    print("Best Solution:", best_solution)
```

In this example, the `evaluate_fitness` function represents the fitness evaluation for a candidate solution. The `generate_random_candidate` function creates a random candidate solution. The `parallel_genetic_algorithm` function runs the genetic algorithm in parallel, evaluating fitness scores using multiprocessing and evolving the population through crossover and mutation.

Replace the fitness evaluation, candidate generation, selection, crossover, and mutation functions with your specific requirements for the optimization problem you are addressing. This is a basic template, and the effectiveness of the genetic algorithm depends on the nature of your optimization problem.



### 12. Parallel Word Count:

**Problem:**
Create a program that counts the occurrences of words in a large text corpus. Use multiprocessing to process different segments of the text concurrently and aggregate the results.

**Explanation:**
Word counting in a large text corpus can be parallelized by dividing the text into segments and processing each segment concurrently. Using multiprocessing, word occurrences can be counted in parallel

This problem involves creating a distributed system for collaborative editing, where multiple users can simultaneously edit a shared document. Here's a simplified example using Python's `socket` module for communication between a server and clients:

**server.py:**
```python
import socket
import threading

class CollaborativeEditorServer:
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.clients = []
        self.lock = threading.Lock()
        self.document = ""

    def start(self):
        server_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        server_socket.bind((self.host, self.port))
        server_socket.listen()

        print(f"Server listening on {self.host}:{self.port}")

        while True:
            client_socket, client_address = server_socket.accept()
            print(f"Accepted connection from {client_address}")
            client_thread = threading.Thread(target=self.handle_client, args=(client_socket,))
            client_thread.start()

    def broadcast(self, message):
        with self.lock:
            for client in self.clients:
                try:
                    client.send(message.encode())
                except:
                    continue

    def handle_client(self, client_socket):
        with self.lock:
            self.clients.append(client_socket)

        while True:
            try:
                data = client_socket.recv(1024).decode()
                if not data:
                    break
                print(f"Received from client: {data}")

                # Handle collaborative editing logic here
                # For simplicity, just append the received data to the document
                with self.lock:
                    self.document += data

                # Broadcast the updated document to all clients
                self.broadcast(self.document)

            except:
                break

        with self.lock:
            self.clients.remove(client_socket)

        client_socket.close()
        print("Connection closed")

if __name__ == "__main__":
    server = CollaborativeEditorServer("127.0.0.1", 5000)
    server.start()
```

**client.py:**
```python
import socket
import threading

class CollaborativeEditorClient:
    def __init__(self, host, port):
        self.host = host
        self.port = port
        self.client_socket = socket.socket(socket.AF_INET, socket.SOCK_STREAM)
        self.username = input("Enter your username: ")

    def start(self):
        self.client_socket.connect((self.host, self.port))

        receive_thread = threading.Thread(target=self.receive_messages)
        receive_thread.start()

        while True:
            message = input()
            if message.lower() == "/exit":
                break
            self.client_socket.send(f"{self.username}: {message}".encode())

        self.client_socket.close()

    def receive_messages(self):
        while True:
            try:
                data = self.client_socket.recv(1024).decode()
                if not data:
                    break
                print(data)
            except:
                break

if __name__ == "__main__":
    client = CollaborativeEditorClient("127.0.0.1", 5000)
    client.start()
```

In this example, the `CollaborativeEditorServer` class handles the server logic, including accepting connections, handling clients in separate threads, and broadcasting changes to all connected clients. The `CollaborativeEditorClient` class represents a simple client that connects to the server, sends messages, and receives updates from other clients.

To run this example, execute the `server.py` script in one terminal window and run the `client.py` script in multiple terminal windows for each client. The clients can simultaneously edit the shared document, and their changes will be broadcasted to all other connected clients.

Please note that this example is minimalistic, and a production-level collaborative editing system would require more sophisticated handling of document changes, user interactions, error handling, and security considerations.

In [9]:
import multiprocessing

def test():
    print("This is my Multiprocessing Program.")

if __name__ == '__main__':
    m = multiprocessing.Process(target=test) 
    print("This is my main program..")
    m.start()
    m.join()   

This is my main program..


In [7]:
test()

This is my Multiprocessing Program.


In [1]:
print("ali")

ali


In [2]:
import multiprocessing

def square(n):
    return n**2

if __name__ == '__main__':
    with multiprocessing.Pool(processes=5) as pool:
        outcome = pool.map(square, [1,25,3,5,3,5,3,74,45,2,4,2])
        print(outcome)   # [1, 625, 9, 25, 9, 25, 9, 5476, 2025, 4, 16, 4]

In [2]:
import multiprocessing 

def producer(q):
    for i in ['ali', 'abbas', 'jawed', 'paikar', 'haider']:
        q.put(i)

def consumer(q):
    while True:
        item = q.get()
        if item is None:
            break
        print(item)

if __name__ == '__main__':
    queue = multiprocessing.Queue()
    m1 = multiprocessing.Process(target=producer, args=(queue,))
    m2 = multiprocessing.Process(target=consumer, args=(queue,))

    m1.start()
    m2.start()
    queue.put("xyz")
    m1.join()
    m2.join()       

# Output
# xyz
# ali
# abbas
# jawed
# paikar
# haider             

In [ ]:
import multiprocessing

def square(index, value):
    value[index] = value[index] ** 2

if __name__ == '__main__':
    arr = multiprocessing.Array('i', [2,3,6,7,8,8,9,3,3,3])
    process = []
    for i in range(10):
        m = multiprocessing.Process(target=square, args=(i, arr))
        process.append(m)
        m.start()
    for m in process:
        m.join()

    print(list(arr))         # [4, 9, 36, 49, 64, 64, 81, 9, 9, 9]

In [7]:
import multiprocessing

def sender(conn, msg):
    for i in msg:
        conn.send(i)
    conn.close()

def receiver(conn):
    while True:
        try:
            msg = conn.recv()
        except Exception as e:
            print(e)
            break
        print(msg) 

if __name__ == '__main__':
    msg = ["My name is Ali Abbas.", "This is my messasge.", "My nickname is Jawed..", "I live in Delhi."]
    parent_conn, child_conn = multiprocessing.Pipe()
    p1 = multiprocessing.Process(target=sender, args=(child_conn, msg))
    p2 = multiprocessing.Process(target=receiver, args=(parent_conn,))
    p1.start()
    p2.start()
    p1.join()
    child_conn.close()
    p2.join()
    parent_conn.close()               

# Output
# My name is Ali Abbas.
# This is my messasge.
# My nickname is Jawed..
# I live in Delhi.    